# DSCI 525 - Web and Cloud Computing

Milestone 2: Your team is planning to migrate to the cloud. AWS gave 400$ (100$ each) to your team to support this. As part of this initiative, your team needs to set up a server in the cloud, a collaborative environment for your team, and later move your data to the cloud. After that, your team can wrangle the data in preparation for machine learning.

## Milestone 2 checklist  
You will have mainly 2 tasks. Here is the checklist...
- To set up a collaborative environment 
    - Setup your EC2 instance with JupyterHub.
    - Install all necessary things needed in your UNIX server (amazon ec2 instance).
    - Set up your S3 bucket.
    - Move the data that you wrangled in your last milestone to s3.
    - To move data from s3.
- Wrangle the data in preparation for machine learning
    - Get the data from S3 in your notebook and make data ready for machine learning.

**Keep in mind:**

- _All services you use are in region us-west-2._

- _Don't store anything in these servers or storage that represents your identity as a student (like your student ID number) ._

- _Use only default VPC and subnet._
    
- _No IP addresses are visible when you provide the screenshot._

- _You do proper budgeting so that you don't run out of credits._ 

- _We want one single notebook for grading, and it's up to your discretion on how you do it. ***So only one person in your group needs to spin up a big instance and a ```t2.xlarge``` is of decent size.***_

- _Please stop the instance when not in use. This can save you some bucks, but it's again up to you and how you budget your money. Maybe stop it if you or your team won't use it for the next 5 hours?

- _Your AWS lab will shut down after 3 hours 30 min. When you start it again, your AWS credentials (***access key***,***secret***, and ***session token***) will change, and you want to update your credentials file with the new one. _

- _Say something went wrong and you want to spin up another EC2 instance, then make sure you terminate the previous one._

- _We will be choosing the storage to be ```Delete on Termination```, which means that stored data in your instance will be lost upon termination. Make sure you save any data to S3 and download the notebooks to your laptop so that next time you have your jupyterHub in a different instance, you can upload your notebook there._

_***Outside of Milestone:*** If you are working as an individual just to practice setting up EC2 instances, make sure you select ```t2.large``` instance (not anything bigger than that as it can cost you money). I strongly recommend you spin up your own instance and experiment with the s3 bucket in doing something (there are many things that we learned and practical work from additional instructions and video series) to get comfortable with AWS. But we won't be looking at it for a grading purpose._

***NOTE:*** Everything you want for this notebook is discussed in lecture 3, lecture 4, and setup instructions.

### 1. Setup your EC2 instance

rubric={correctness:20}

#### Please attach this screen shots from your group for grading.

![](screencap/EC2-group1.png)

### 2. Setup your JupyterHub

rubric={correctness:20}

#### Please attach this screen shots from your group for grading

![](screencap/jupyter-group1.png)

### 3. Setup the server 

rubric={correctness:20}

3.1) Add your team members to EC2 instance.

3.2) Setup a common data folder to download data, and this folder should be accessible by all users in the JupyterHub.
    
3.3)(***OPTIONAL***) Setup a sharing notebook environment.

3.4) Install and configure AWS CLI.

#### Please attach this screen shots from your group for grading

Make sure you mask the IP address refer [here](https://www.anysoftwaretools.com/blur-part-picture-mac/).

![](screencap/shared_folder-group1.png)

### 4. Get the data what we wrangled in our first milestone. 

You have to install the packages that are needed. Refer this TLJH [document]( https://tljh.jupyter.org/en/latest/howto/env/user-environment.html).Refer ```pip``` section.

Don't forget to add option -E. This way, all packages that you install will be available to other users in your JupyterHub.
These packages you must install and install other packages needed for your wrangling.

    sudo -E pip install pandas
    sudo -E pip install pyarrow
    sudo -E pip install s3fs

As in the last milestone, we looked at getting the data transferred from Python to R, and we have different solutions. Henceforth, I uploaded the parquet file format, which we can use moving forward.

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

Rememeber here we gave the folder that we created in Step 3.2 as we made it available for all the users in a group.

In [5]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/Users/stevenprivate/mds/525/web_and_cloud_group_1/data/my_shared_data_folder/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

In [8]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [9]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

### 5. Setup your S3 bucket and move data

rubric={correctness:20}

5.1)  Create a bucket name should be mds-s3-xxx. Replace xxx with your "groupnumber".

5.2)  Create your first folder called "output".

5.3) Move the "observed_daily_rainfall_SYD.csv" file from the Milestone1 data folder to your s3 bucket from your local computer.

5.4) Moving the parquet file we downloaded(combined_model_data_parti.parquet) in step 4 to S3 using the cli what we installed in step 3.4.

#### Please attach this screen shots from your group for grading

Make sure it has 3 objects.

https://github.ubc.ca/mds-2021-22/DSCI_525_web-cloud-comp_students/blob/master/release/milestone2/image/4_result.png

### 6. Wrangle the data in preparation for machine learning

rubric={correctness:20}

Our data currently covers all of NSW, but say that our client wants us to create a machine learning model to predict rainfall over Sydney only. There's a bit of wrangling that needs to be done for that:
1. We need to query our data for only the rows that contain information covering Sydney
2. We need to wrangle our data into a format suitable for training a machine learning model. That will require pivoting, resampling, grouping, etc.

To train an ML algorithm we need it to look like this:

||model-1_rainfall|model-2_rainfall|model-3_rainfall|...|observed_rainfall|
|---|---|---|---|---|---|
|0|0.12|0.43|0.35|...|0.31|
|1|1.22|0.91|1.68|...|1.34|
|2|0.68|0.29|0.41|...|0.57|

ob = pd.read_csv('s3://mds-s3-21/observed_daily_rainfall_SYD.csv', storage_options = aws_credentials)
ob["model"] = "observed_rainfall"6.1) Get the data from s3 (```combined_model_data_parti.parquet``` and ```observed_daily_rainfall_SYD.csv```)

6.2) First query for Sydney data and then drop the lat and lon columns (we don't need them).
```
syd_lat = -33.86
syd_lon = 151.21
```
Expected shape ```(1150049, 2)```.

6.3) Save this processed file to s3 for later use:

  Save as a csv file ```ml_data_SYD.csv``` to ```s3://mds-s3-xxx/output/```
  expected shape ```(46020,26)``` - This includes all the models as columns and also adding additional column ```Observed``` loaded from ```observed_daily_rainfall_SYD.csv``` from s3.

## I copied the code from above and changed a bit for running locally. Feel free to delete reading code once you transfer things to S3. And timing information in available below

In [63]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [64]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [65]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

In [66]:
%%time
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.96 s, sys: 5.72 s, total: 8.68 s
Wall time: 2min 59s


Output:
```
CPU times: user 2.96 s, sys: 5.72 s, total: 8.68 s
Wall time: 2min 59s
```

In [67]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 2.97 s, sys: 360 ms, total: 3.33 s
Wall time: 3.62 s


Output:
```
CPU times: user 2.97 s, sys: 360 ms, total: 3.33 s
Wall time: 3.62 s
```

## The following code should be good for S3 in terms of reading data. But did not test, feel free to edit it

In [ ]:
# This should be the S3 version!

with open('/srv/data/shared_data/credential.json') as f:
    aws_credentials = json.load(f)
    
rainfall = pd.read_parquet('s3://mds-s3-1/combined_model_data_parti.parquet', 
                     storage_options = aws_credentials)

In [10]:
# This should be the S3 version!

# with open('/srv/data/shared_data/credential.json') as f:
#     aws_credentials = json.load(f)
    
rainfall = pd.read_parquet('s3://mds-s3-001/combined_model_data_parti.parquet')

FSTimeoutError: 

In [ ]:
observed_rainfall = pd.read_csv('s3://mds-s3-1/observed_daily_rainfall_SYD.csv', 
                                parse_dates=["time"], 
                                storage_options = aws_credentials)
observed_rainfall.set_index('time', inplace=True)
observed_rainfall.head()

In [15]:
%%time
observed_rainfall = pd.read_csv('s3://mds-s3-001/observed_daily_rainfall_SYD.csv', 
                                parse_dates=["time"])
observed_rainfall.set_index('time', inplace=True)
observed_rainfall.head()

CPU times: user 45.7 ms, sys: 35.5 ms, total: 81.2 ms
Wall time: 1.52 s


,rain (mm/day)
time,
1889-01-01,0.006612
1889-01-02,0.090422
1889-01-03,1.401452
1889-01-04,14.869798
1889-01-05,0.467628


Output (time for reading CSV file from S3):
```
CPU times: user 45.7 ms, sys: 35.5 ms, total: 81.2 ms
Wall time: 1.52 s
```

## The following code just for wrangling data locally

In [11]:
%%time
rainfall = pd.read_parquet(
    "../data/my_shared_data_folder/combined_model_data_parti.parquet")
rainfall.head()

CPU times: user 3.25 s, sys: 6.42 s, total: 9.68 s
Wall time: 4.13 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


Output (time for reading parquet file from local disk):
```
CPU times: user 3.25 s, sys: 6.42 s, total: 9.68 s
Wall time: 4.13 s
```

In [12]:
# URI for csv file: s3://mds-s3-001/observed_daily_rainfall_SYD.csv


In [14]:
observed_rainfall = pd.read_csv('../data/observed_daily_rainfall_SYD.csv', parse_dates=["time"])
observed_rainfall.set_index('time', inplace=True)
observed_rainfall.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/observed_daily_rainfall_SYD.csv'

In [16]:
sydney_rainfall = rainfall.query("lat_min <= -33.86 and lat_max >= -33.86 and lon_min <= 151.21 and lon_max >= 151.21")
sydney_rainfall = sydney_rainfall[['time', 'rain (mm/day)', 'model']]
sydney_rainfall['time'] = pd.to_datetime(sydney_rainfall["time"]).dt.date
sydney_rainfall = sydney_rainfall.set_index('time')
sydney_rainfall.head()

,rain (mm/day),model
time,,
1889-01-01,0.040427,ACCESS-CM2
1889-01-02,0.073777,ACCESS-CM2
1889-01-03,0.232656,ACCESS-CM2
1889-01-04,0.911319,ACCESS-CM2
1889-01-05,0.698013,ACCESS-CM2


In [17]:
sydney_rainfall.shape

(1150049, 2)

In [18]:
observed_rainfall.shape

(46020, 1)

In [19]:
# Pivot
pivot_sydney_rainfall = (sydney_rainfall.reset_index().pivot(index='time', 
                                                             columns='model', 
                                                             values = 'rain (mm/day)').rename_axis(None, axis=1))

pivot_sydney_rainfall.head()

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MIROC6,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,0.648031,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,1.099883,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,3.931751,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,0.627365,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,0.019777,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335


In [20]:
pivot_sydney_rainfall["observed_rainfall"] = observed_rainfall["rain (mm/day)"]
pivot_sydney_rainfall.head()

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,observed_rainfall
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,35.579336,4.268112e+00,1.107466e-03,11.410537,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520,1.190141e+00,1.015323e-04,4.014984,1.312700e+00,0.946211,2.788724,4.771375,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467,1.003845e-09,1.760345e-05,9.660565,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624,8.225225e-02,1.808932e-01,3.951528,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472226e-07,5.301038e-01,0.002699,2.185454,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686,2.496841e+00,4.708019e-09,2.766362,1.822940e+01,0.339267,0.002468,11.920356,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628


In [21]:
pivot_sydney_rainfall.shape

(46020, 26)

How the final file format looks like
https://github.ubc.ca/mds-2021-22/DSCI_525_web-cloud-comp_students/blob/master/release/milestone2/image/finaloutput.png

Shape ```(46020,26 )```

(***OPTIONAL***) If you are interested in doing some benchmarking!! How much time it took to read..
- Parquet file from your local disk ?
- Parquet file from s3 ?
- CSV file from s3 ?
    For that, upload the CSV file (```combined_model_data.csv```
     )to S3 and try to read it instead of parquet. 

#### Reading time 

| Method |Time taken |
|:-----------:|:----------:|
| Parquet file from your local disk  | 2min 59s |
| Parquet file from s3    |     |
| CSV file from s3|             |
  